In [1]:
# !pip install tensorflow-gpu==2.3
# !pip install sklearn
# !pip install matplotlib

In [2]:
import numpy as np
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
import os
import matplotlib.pyplot as plt

In [3]:
import tensorflow
import tensorflow as tf

In [4]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [5]:
file_dir1 = 'C:/Users/BVer/are_you_ok/preprocessing'

In [6]:
raw_data = np.load(file_dir1 + '/p_video/' + 'raw_kicking_k1.mp4_.npy')

In [7]:
raw_data.shape

(47,)

In [8]:
raw_data[0][0][8]

0.0

In [9]:
from tensorflow.keras.models import load_model

right_arm_model = load_model('./models/right_arm_model.h5')
left_arm_model = load_model('./models/left_arm_model.h5')
right_leg_model = load_model('./models/right_leg_model.h5')
left_leg_model = load_model('./models/left_leg_model.h5')

In [10]:
def pred_multiple(raw_data):
    position = []
    last_position = []
    detected_humans = [] #[human][index, part_seq, part_nan_count, part_action, count]
    
    for object_at_frame in range(len(raw_data)): #프레임 단위 반복
        last_position += position
        position = []
        for object_at in range(len(raw_data[object_at_frame])): #사람 단위 반복
            detect_state = 0
            for detect_seq in detected_humans: #사람이 전 프레임에 있던 사람인지 확인
                if raw_data[object_at_frame][object_at][8] == detect_seq[0]: #해당 인덱스의 사람이 존재하는지 검사
                    detect_seq[13] = 0
                    position.append(detect_seq[0])
                    last_position.remove(detect_seq[0])
                    pred_raw_part(raw_data[object_at_frame][object_at], detect_seq)
                    detect_state = 1
                    break
                    
            if detect_state == 0:
                detected_humans.append([raw_data[object_at_frame][object_at][8], [], 0, 0, [], 0, 0, [], 0, 0, [], 0, 0, 0])
                position.append(raw_data[object_at_frame][object_at][8])
                pred_raw_part(raw_data[object_at_frame][object_at], detected_humans[-1])
                
        del_list = []         
        for gone_position in last_position:
            for gone_index in detected_humans:
                if gone_index[0] == gone_position:
                    gone_index[13] += 1
                    if gone_index[13] > 10:
                        del_list.append(gone_position)
                    break
            
        if len(del_list) > 0:
            for del_position in range(len(del_list)):
                for del_index in detected_humans:
                    if del_index[0] == del_list[del_position]:
                        last_position.remove(del_list[del_position])
                        detected_humans.remove(del_index)
                        break
                
               

In [11]:
def pred_raw_part(object_n, human_seq):
    object_n = object_n[:-1]
    seq_length = 30

    actions = ['assult', 'normal']
    
    nan_skip_count = 10 #오차 허용 프레임
    
    for process_count in range(4):
        human_seq[process_count * 3 + 2] = take_not_nan(object_n[process_count * 2], object_n[process_count * 2 + 1], human_seq[process_count * 3 + 1], human_seq[process_count * 3 + 2])
        
    for process_count in range(4):
        if human_seq[process_count * 3 + 2] == 15:
            print("cleared seq")
            human_seq[process_count * 3 + 1] = []
            human_seq[process_count * 3 + 3] = 0
            
    for process_count, model in enumerate([right_arm_model, left_arm_model, right_leg_model, left_leg_model]):
        human_seq[process_count * 3 + 3] = pred_part(human_seq[process_count * 3 + 1], human_seq[process_count * 3 + 2], model, human_seq[process_count * 3 + 3])

In [12]:
def take_not_nan(key1, key2, part_seq, part_nan):
    if not np.isnan(key1) and not np.isnan(key2):
        part_seq.append([key1, key2])
        return 0
    else:
        part_nan += 1
        if part_nan > 10: #오차 허용 프레임
            return 15
        else:
            return part_nan
        
def pred_part(part_seq, part_nan, part_model, part_action):
    seq_length = 30
    if len(part_seq) > seq_length and part_nan == 0:
        input_data = np.expand_dims(np.array(part_seq[-seq_length:], dtype=np.float32), axis=0)
        y_pred = part_model.predict(input_data).squeeze()

        i_pred = int(np.argmax(y_pred))
        conf = y_pred[i_pred]
        
#         print(conf, i_pred, part_name)

        if conf > 0.5 and i_pred == 0: #신뢰도
            part_action += 1
            print(part_action)
            if part_action > 2:
                print('detect  assult ', conf)
            return part_action
        else:
            return 0
    return part_action

In [13]:
import cv2
t0 = cv2.getTickCount()
pred_multiple(raw_data)
t1 = cv2.getTickCount()
print ("took %f seconds." % ((t1-t0)/cv2.getTickFrequency()))

took 4.172504 seconds.
